In [42]:
import cv2
import dlib
import os
from PIL import Image
import numpy as np

In [43]:
face_detector= dlib.get_frontal_face_detector()
points_detector= dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

image= cv2.imread('people2.jpg')

face_detection= face_detector(image, 1)
for face in face_detection:
    points= points_detector(image, face)
    for point in points.parts():
        cv2.circle(image, (point.x, point.y), 2, (0,255,0), 1)

    l,t,r,b= face.left(),face.top(), face.right(), face.bottom()
    cv2.rectangle(image, (l,t), (r,b), (0,255,0),2)
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
face_detector= dlib.get_frontal_face_detector()
points_detector= dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_descriptor_extractor= dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

index= {}
idx=0
face_descriptors= None

paths= [os.path.join('yalefaces/train',f) for f in os.listdir('yalefaces/train')]
for path in paths:
    image= Image.open(path).convert('RGB')
    image_np= np.array(image, 'uint8')
    face_detection = face_detector(image_np,1)
    for face in face_detection:
        l,t,r,b= face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(image_np, (l,t), (r,b), (0,255,0), 2)

        points= points_detector(image_np, face)
        for point in points.parts():
            cv2.circle(image_np, (point.x,point.y), 2, (0,255,0), 1)

        face_descriptor= face_descriptor_extractor.compute_face_descriptor(image_np, points)
        face_descriptor= [f for f in face_descriptor]
        face_descriptor= np.asarray(face_descriptor, dtype=np.float64)
        face_descriptor= face_descriptor[np.newaxis, :]
        if face_descriptors is None:
            face_descriptors= face_descriptor
        else:
            face_descriptors= np.concatenate((face_descriptors, face_descriptor), axis=0)
        index[idx]= path
        idx+=1
    
    # cv2.imshow('Image',image_np)
    # cv2.waitKey(delay = 1)
    # cv2.destroyAllWindows()

In [ ]:
np.linalg.norm(face_descriptors[103]- face_descriptors, axis=1)

In [63]:
threshold= 0.5
predictions= []
expected_outputs= []

paths= [os.path.join('yalefaces/test',f )for f in os.listdir('yalefaces/test')]
for path in paths:
    image= Image.open(path).convert('RGB')
    image_np= np.array(image, 'uint8')
    face_detection= face_detector(image_np, 1)
    for face in face_detection:
        points= points_detector(image_np, face)
        face_descriptor= face_descriptor_extractor.compute_face_descriptor(image_np, points)
        face_descriptor= [f for f in face_descriptor]
        face_descriptor= np.asarray(face_descriptor, dtype= np.float64)
        face_descriptor= face_descriptor[np.newaxis, :]

        distances= np.linalg.norm(face_descriptor- face_descriptors,axis=1 )
        min_index= np.argmin(distances)
        min_distance= distances[min_index]
        
        if min_distance<=threshold:
            name_pred= int(os.path.split(index[min_index])[1].split('.')[0].replace('subject',''))
        else:
            name_pred= 'Not Identified'

        name_real= int(os.path.split(path)[1].split('.')[0].replace('subject',''))
        predictions.append(name_pred)
        expected_outputs.append(name_real)

        cv2.putText(image_np, 'Pred: '+ str(name_pred), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
        cv2.putText(image_np, 'Pred: '+ str(name_real), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))

predictions =np.array(predictions)
expected_outputs= np.array(expected_outputs)
print(predictions)
print(expected_outputs)
    
    # cv2.imshow('Image', image_np)
    # cv2.waitKey(delay= 1)
    # cv2.destroyAllWindows()

[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12 12
 13 13 14 14 15 15]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12 12
 13 13 14 14 15 15]


In [64]:
from sklearn.metrics import accuracy_score, confusion_matrix 
acc= accuracy_score(expected_outputs, predictions)
print(acc)

1.0
